In [1]:
import torch
from gpytorch.means import ConstantMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import WhitenedVariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import AbstractVariationalGP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood

In [2]:
class NegativeKLDivergence(AddedLossTerm):
    def __init__(self, variational_strategy):
        self.variational_strategy = variational_strategy
    
    def loss(self):
        return -1 * self.variational_strategy.kl_divergence().sum()

class HiddenGPLayer(AbstractVariationalGP):
    """
    Represents a hidden layer in a deep GP where inference is performed via the doubly stochastic method of 
    Salimbeni et al., 2017. Upon calling, instead of returning a variational distribution q(f), returns samples
    from the variational distribution. 
    
    See the documentation for __call__ below for more details below. Note that the behavior of __call__
    will change to be much more elegant with multiple batch dimensions; however, the interface doesn't really
    change.
    
    Args:
        - input_dims (int): Dimensionality of input data expected by each GP
        - output_dims (int): Number of GPs in this layer, equivalent to output dimensionality.
        - num_inducing (int): Number of inducing points for this hidden layer
        - num_samples (int): Number of samples to draw from q(f) for returning
    """
    def __init__(self, input_dims, output_dims, num_inducing=512, num_samples=20):
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_samples = num_samples
        inducing_points = torch.randn(output_dims, num_inducing, input_dims)
        
        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_size=output_dims
        )
        variational_strategy = WhitenedVariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )
        
        super(HiddenGPLayer, self).__init__(variational_strategy)
        
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(RBFKernel(batch_size=output_dims), batch_size=output_dims)
        
        self.register_added_loss_term("hidden_kl_divergence")
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
    
    def __call__(self, inputs):
        """
        Forward data through this hidden GP layer.
        
        If the input is 2 dimensional, we pass the input through each hidden GP, resulting in a `h x n` batch
        Gaussian distribution. We then draw `s` samples from these Gaussians and reshape the result to be
        `s x n x h` (e.g., h becomes the dimensionality of the output).
        
        If the input is 3 dimensional, we assume that the input is `s x n x d`, e.g. the batch dimension
        corresponds to the number of samples. We use this as the number of samples to draw, and just propagate
        each sample through the hidden layer. The output will be `s x n x h`. 
        
        If the input is 4 dimensional, we assume that for some reason the user has already reshaped things to be
        `h x s x n x d`. We reshape this internally to be `h x sn x d`, and the output will be `s x n x h`.
        
        The goal of these last two points is that if you have a tensor `x` that is `n x d`, then:
            >>> hidden_gp2(hidden_gp(x))

        will just work, and return a tensor of size `s x n x h2`, where `h2` is the output dimensionality of 
        hidden_gp2. In this way, hidden GP layers are easily composable.
        """
        # TODO: Simplify this logic once multiple batch dimensions is supported
        
        if inputs.dim() == 2:
            # Assume new input entirely
            inputs = inputs.unsqueeze(0)
            inputs = inputs.expand(self.output_dims, inputs.size(-2), self.input_dims)
        elif inputs.dim() == 3:
            # Assume batch dim is samples, not output_dim
            inputs = inputs.unsqueeze(0)
            inputs = inputs.expand(self.output_dims, inputs.size(1), inputs.size(-2), self.input_dims)

        if inputs.dim() == 4:
            num_samples = inputs.size(-3)
            inputs = inputs.view(self.output_dims, inputs.size(-2) * inputs.size(-3), self.input_dims)
            reshape_output = True
        else:
            reshape_output = False
            num_samples = self.num_samples

        variational_dist_f = super(HiddenGPLayer, self).__call__(inputs)
        mean_qf = variational_dist_f.mean
        std_qf = variational_dist_f.variance.sqrt()
        
        if reshape_output:
            samples = torch.distributions.Normal(mean_qf, std_qf).rsample()
            samples = samples.view(self.output_dims, num_samples, -1).permute(1, 2, 0)
        else:
            samples = torch.distributions.Normal(mean_qf, std_qf).rsample(torch.Size([num_samples]))
            samples = samples.transpose(-2, -1)
        
        loss_term = NegativeKLDivergence(self.variational_strategy)
        self.update_added_loss_term("hidden_kl_divergence", loss_term)
        
        return samples

In [3]:
class DeepGP(AbstractVariationalGP):
    def __init__(self, input_dims, hidden_dims, output_dims, num_inducing=512, num_samples=20):
        self.input_dims = input_dims
        self.hidden_dims = hidden_dims
        self.output_dims = output_dims
        self.num_samples = num_samples
    
        inducing_points = torch.randn(output_dims, num_inducing, hidden_dims)

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_size=output_dims
        )

        variational_strategy = WhitenedVariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )
        
        super(DeepGP, self).__init__(variational_strategy)
        
        self.mean_module = ConstantMean(batch_size=output_dims)
        self.covar_module = ScaleKernel(RBFKernel(batch_size=output_dims), batch_size=output_dims)
        
        # For more layers, just make more hidden layers to forward through
        self.hidden_gp_layer = HiddenGPLayer(
            input_dims=input_dims,
            output_dims=hidden_dims,
            num_inducing=num_inducing,
            num_samples=num_samples
        )
    
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
    
    def __call__(self, inputs):
        """
        The call method of a Deep GP differs from a standard variational GP simply in that we first
        pass the data through any hidden GP layers, reshape the input to account for the fact that
        the batch dimension should be interpreted as a number of samples, and then pass through the final layer.
        
        In some sense, the DeepGP object itself represents the last layer of the deep GP.
        """
        # Forward through more layers here if they exist
        hidden_inputs = self.hidden_gp_layer(inputs)
        
        # hidden_inputs is num_samples x n_inputs x hidden_dims
        # Combine samples and inputs dimension since we want to mean over those in the likelihood
        last_layer_inputs = hidden_inputs.contiguous().view(-1, self.hidden_dims)
        last_layer_inputs = last_layer_inputs.unsqueeze(0).expand(self.output_dims, self.num_samples * inputs.size(-2), self.hidden_dims)
        
        return super(DeepGP, self).__call__(last_layer_inputs)

In [4]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor
import numpy as np

if not os.path.isfile('elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', 'elevators.mat')
    
data = torch.Tensor(loadmat('elevators.mat')['data'])
X = data[:, :-1]
y = data[:, -1]

N = data.shape[0]
np.random.seed(0)
data = data[np.random.permutation(np.arange(N)),:]

train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

mean = train_x.mean(dim=-2, keepdim=True)
std = train_x.std(dim=-2, keepdim=True) + 1e-6
train_x = (train_x - mean) / std
test_x = (test_x - mean) / std

mean,std = train_y.mean(),train_y.std()
train_y = (train_y - mean) / std
test_y = (test_y - mean) / std

In [5]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

In [19]:
model = DeepGP(input_dims=18, hidden_dims=15, output_dims=1, num_inducing=512, num_samples=1).cuda()
likelihood = GaussianLikelihood().cuda()
mll = VariationalELBO(likelihood, model, train_x.size(-2))

In [ ]:
num_epochs = 40

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.001)

for i in range(num_epochs):
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(x_batch)
        train_y_reshaped = y_batch.unsqueeze(0).expand(model.num_samples, *y_batch.shape).contiguous().view(model.output_dims, model.num_samples * y_batch.size(-1))
        loss = -mll(output, train_y_reshaped).sum()
        
        print('Epoch %d [%d/%d] - Loss: %.3f' % (i + 1, minibatch_i, len(train_loader), loss.item()))

        loss.backward()
        optimizer.step()

In [24]:
preds = likelihood(model(test_x))

In [25]:
torch.sqrt(torch.mean(torch.pow(preds.mean.reshape(model.num_samples, -1).mean(0) - test_y, 2)))

tensor(0.3520, device='cuda:0', grad_fn=<SqrtBackward>)